In [ ]:
pip install folium

In [ ]:
pip install dash

In [ ]:
pip install wget

In [ ]:
import folium 
import wget
import pandas as pd 

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
app = dash.Dash(__name__)


In [ ]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df = pd.read_csv(spacex_csv_file) 

In [ ]:
spacex_df = spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_sites_df = spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_sites_df = launch_sites_df[["Launch Site", "Lat", "Long"]]
launch_sites_df

In [ ]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)


In [ ]:
circle = folium.Circle(nasa_coordinate, radius=2000, color= '#d35400', fill=True).add_child(folium.Popup("NASA Johnson Space Center"))

marker = folium.map.Marker(
    nasa_coordinate, 

    icon= DivIcon( 
        icon_size=(20,20),
        icon_anchor=(0,0), 
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )

    )

site_map.add_child(circle)
site_map.add_child(marker)



In [ ]:
# Reinitialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add a circle and marker for each launch site in the launch_sites_df dataframe
for index, row in launch_sites_df.iterrows():
    # Extract latitude and longitude
    site_coordinate = [row['Lat'], row['Long']]
    
    # Create a circle object for the launch site
    circle = folium.Circle(site_coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(row['Launch Site']))
    
    # Create a marker object for the launch site
    marker = folium.map.Marker(
        site_coordinate,
        icon=DivIcon(icon_size=(20,20), icon_anchor=(0,0), 
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'])
    )
    
    # Add the circle and marker objects to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map


In [ ]:
spacex_df.tail()

In [ ]:
marker_cluster = MarkerCluster()

In [ ]:
spacex_df["marker_color"] = spacex_df["class"].apply(lambda x: "green" if x == 1 else "red")
spacex_df.head() 

"""
Yukarıdaki Kodun Daha Geniş Hali 

def assign_marker_color(launch_outcome): 
    if launch_outcome == 1:
        return "green"
    else: 
        return "red"
    
spacex_df["marker_color"] = spacex_df["class"].apply(assign_marker_color)
spacex_df.tail()

"""

In [ ]:
marker_cluster = MarkerCluster()

site_map.add_child(marker_cluster)

for index, record in spacex_df.iterrows(): 
    marker = folium.Marker(
        location=[record["Lat"], record["Long"]], 
        icon=folium.Icon(color="white", icon_color=record["marker_color"])
    )
    marker_cluster.add_child(marker)
    
site_map

In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [ ]:
def calculate(lat1, lon1, lat2, lon2): 

    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distance = R * c
    return distance



In [ ]:
# Eğer birden fazla satır varsa, belirli bir satırı seçmek için .iloc metodunu kullanabiliriz.
# Örnek olarak, ilk satırı seçelim:
launch_sites_lat = spacex_df["Lat"].iloc[0] 
launch_sites_lon = spacex_df["Long"].iloc[0]

# MousePosition ile belirlenen kıyı şeridinin koordinatları
coastline_lat = 28.56367
coastline_lon = -80.56802

# İki nokta arasındaki mesafeyi hesaplama
distance_coastline = calculate(launch_sites_lat, launch_sites_lon, coastline_lat, coastline_lon)

print(f"The distance between launch site and coastline is approximately {distance_coastline:.2f} kilometers.")

site_map = folium.Map(location=[launch_sites_lat, launch_sites_lon], zoom_start=14)

points = ([launch_sites_lat,launch_sites_lon], [coastline_lat, coastline_lon])
line = folium.PolyLine(locations=points, colour="blue", weight=2.5, opacity=1).add_child(folium.Popup(f"{distance_coastline:.2f} km"))

site_map.add_child(line)
site_map
